## Testing Geoapify Places API

In [1]:
import sys
# change working directory to the root of the project
sys.path.append('../')
from src.methods.data_ingestion import call_api

In [29]:
request_params = {
    "apiKey": "493bbfea1f124b27906cf052aa330680",
    "categories": "entertainment.museum",
    "limit": "20",
}
filters = {
        "circle": "-1.257677,51.7520131,5000",
}

url = 'https://api.geoapify.com/v2/places?'

# append parameters to the url
for key, value in request_params.items():
    url = url.replace('?', f'?{key}={value}&')

# append filters to the url
for key, value in filters.items():
    url = url.replace('?', f'?filter={key}:{value}&')

# remove the last '&' from the url
url = url[:-1]
url


'https://api.geoapify.com/v2/places?filter=circle:-1.257677,51.7520131,5000&limit=20&categories=entertainment.museum&apiKey=493bbfea1f124b27906cf052aa330680'

Find all museums in a 5000m radium from Oxford city centre

In [30]:
res = call_api(url)
res